### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']   # Remove the first index column
del reviews['Unnamed: 0']  # Remove the first index column

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [2]:
reviews.head(2)

,user_id,movie_id,rating,timestamp,date
0,1,114508,8,1381006850,2013-10-05 21:00:50
1,2,208092,5,1586466072,2020-04-09 21:01:12


In [3]:
reviews.head(2)

,user_id,movie_id,rating,timestamp,date
0,1,114508,8,1381006850,2013-10-05 21:00:50
1,2,208092,5,1586466072,2020-04-09 21:01:12


In [4]:
b=dict(list(reviews.groupby('movie_id')['date']))
b[67355]  # get list for 'movie_id'=67335

259341    2013-09-17 19:30:06
469230    2017-06-05 07:20:51
477023    2018-12-31 09:42:01
645192    2015-02-24 03:05:27
731197    2016-06-03 17:15:36
Name: date, dtype: object

In [5]:
b=reviews.groupby('movie_id')
b.get_group(67355)

,user_id,movie_id,rating,timestamp,date
259341,20843,67355,8,1379446206,2013-09-17 19:30:06
469230,37018,67355,10,1496647251,2017-06-05 07:20:51
477023,37629,67355,9,1546249321,2018-12-31 09:42:01
645192,50265,67355,10,1424747127,2015-02-24 03:05:27
731197,57381,67355,10,1464974136,2016-06-03 17:15:36


In [25]:
ratings=pd.DataFrame(reviews.groupby('movie_id').rating.mean())

ratings['Num_rating']=reviews.groupby('movie_id').rating.count().values
ratings['recent_date']=reviews.groupby('movie_id').date.max().values
    
ratings.sort_values(by=['rating','Num_rating','recent_date'],ascending=False,inplace=True)
ratings=ratings[ratings['Num_rating']>=5]
## top_movies.drop(top_movies[top_movies['Num_rating']<5].index,inplace=True)
print(ratings.shape)
ratings.head(10)

(11536, 3)


,rating,Num_rating,recent_date
movie_id,,,
4921860,10.0,48,2016-08-14 17:16:50
5262972,10.0,28,2016-01-08 00:44:43
6662050,10.0,22,2019-04-20 22:29:19
2737018,10.0,10,2015-05-10 22:56:01
5804314,10.0,7,2019-12-25 16:27:47
2560840,10.0,6,2016-01-23 00:30:44
2219210,10.0,6,2013-07-22 23:30:52
9882084,10.0,5,2019-06-29 03:37:46
4448444,10.0,5,2017-09-09 13:51:48


In [34]:
movies_rec=ratings.join(movies.set_index('movie_id'),on='movie_id')
movies_rec.head(3)

,rating,Num_rating,recent_date,movie,genre,date,1800's,1900's,2000's,Action,...,Sport,Mystery,Romance,Music,Adventure,Crime,Game-Show,Musical,Family,News
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,10.0,48,2016-08-14 17:16:50,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,2015,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5262972,10.0,28,2016-01-08 00:44:43,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6662050,10.0,22,2019-04-20 22:29:19,Five Minutes (2017),Short|Comedy,2017,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def create_ranked_df(movies, reviews):
        '''
        INPUT
        movies - the movies dataframe
        reviews - the reviews dataframe
        
        OUTPUT
        ranked_movies - a dataframe with movies that are sorted by highest avg rating, more reviews, 
                        then time, and must have more than 4 ratings
        '''
        
        # Pull the average ratings and number of ratings for each movie
        movie_ratings = reviews.groupby('movie_id')['rating']
        avg_ratings = movie_ratings.mean()
        num_ratings = movie_ratings.count()
        last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['date'])
        last_rating.columns = ['last_rating']

        # Add Dates
        rating_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings})
        rating_count_df = rating_count_df.join(last_rating)

        # merge with the movies dataset
        movie_recs = movies.set_index('movie_id').join(rating_count_df)

        # sort by top avg rating and number of ratings
        ranked_movies = movie_recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

        # for edge cases - subset the movie list to those with only 5 or more reviews
        ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
        
        return ranked_movies
    

def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time

    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''

    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies
        



Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [9]:
ranked_movies = create_ranked_df(movies, reviews) # only run this once - it is not fast

In [24]:
ranked_movies.head(3)

,movie,genre,date,1800's,1900's,2000's,Action,History,Short,Biography,...,Music,Adventure,Crime,Game-Show,Musical,Family,News,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,2015,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.0,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,10.0,28,2016-01-08 00:44:43
6662050,Five Minutes (2017),Short|Comedy,2017,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,10.0,22,2019-04-20 22:29:19


In [10]:
popular_recommendations('', 5, ranked_movies)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Five Minutes (2017)',
 'Selam (2013)',
 'Let There Be Light (2017)']

In [35]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations('', 20, movies_rec) # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations('53968', 5, ranked_movies)# Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations('7000', 100, ranked_movies)# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations('43', 35, movies_rec)# Your solution list here



In [36]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [21]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
    '''
    REDO THIS DOC STRING
    
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Filter movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]

    if genres is not None:
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]
            
            
    # create top movies list 
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies


In [20]:
ranked_movies[['Action','Family']].sum(axis=1)[0:3]

movie_id
4921860    0
5262972    0
6662050    0
dtype: int64

In [22]:
# Top 20 movies recommended for id 1 with years=['2015', '2016', '2017', '2018'], genres=['History']
recs_20_for_1_filtered = popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

# Top 5 movies recommended for id 53968 with no genre filter but years=['2015', '2016', '2017', '2018']
recs_5_for_53968_filtered = popular_recs_filtered('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018'])

# Top 100 movies recommended for id 70000 with no year filter but genres=['History', 'News']
recs_100_for_70000_filtered = popular_recs_filtered('70000', 100, ranked_movies, genres=['History', 'News'])


In [23]:
### You Should Not Need To Modify Anything In This Cell

# check 1 
assert t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History']) == recs_20_for_1_filtered,  "The first check failed..."
# check 2
assert t.popular_recs_filtered('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018']) == recs_5_for_53968_filtered,  "The second check failed..."
# check 3
assert t.popular_recs_filtered('70000', 100, ranked_movies, genres=['History', 'News']) == recs_100_for_70000_filtered,  "The third check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!
